Indeed, in many situations you don't really need to know the implementation details. However, having a good understanding of how thins work can help you quickly home in on the appropriate model, the right training alogrithm to use, and a good set of hyperparameters for your task.

